In [1]:
import boto3

In [2]:
client = boto3.client('ec2')

In [3]:
vpc = client.describe_vpcs()

In [4]:
vpc.keys()

dict_keys(['Vpcs', 'ResponseMetadata'])

In [5]:
vpc['Vpcs']

[{'CidrBlock': '172.31.0.0/16',
  'CidrBlockAssociationSet': [{'AssociationId': 'vpc-cidr-assoc-2d8a1c46',
    'CidrBlock': '172.31.0.0/16',
    'CidrBlockState': {'State': 'associated'}}],
  'DhcpOptionsId': 'dopt-9f21fbf9',
  'InstanceTenancy': 'default',
  'IsDefault': True,
  'State': 'available',
  'VpcId': 'vpc-cc3740aa'}]

In [6]:
vpc['Vpcs'][0]['VpcId']

'vpc-cc3740aa'

In [7]:
images = client.describe_images(Owners=['self'])

In [8]:
images['Images'][1]

{'Architecture': 'x86_64',
 'BlockDeviceMappings': [{'DeviceName': '/dev/sda1',
   'Ebs': {'DeleteOnTermination': True,
    'Encrypted': False,
    'SnapshotId': 'snap-00985465f1a74f668',
    'VolumeSize': 100,
    'VolumeType': 'standard'}}],
 'CreationDate': '2018-06-01T17:35:11.000Z',
 'Description': '[Copied ami-2dc6f954 from eu-west-1] cpu_with_keras_retinanet',
 'EnaSupport': True,
 'Hypervisor': 'xen',
 'ImageId': 'ami-051fd589759f16b6c',
 'ImageLocation': '272317939526/cpu_with_keras_retinanet_mine',
 'ImageType': 'machine',
 'Name': 'cpu_with_keras_retinanet_mine',
 'OwnerId': '272317939526',
 'Public': False,
 'RootDeviceName': '/dev/sda1',
 'RootDeviceType': 'ebs',
 'SriovNetSupport': 'simple',
 'State': 'available',
 'VirtualizationType': 'hvm'}

In [9]:
[ (image['ImageId'], image['Name']) for image in images['Images'] if 'gpu' in image['Name'] ]

[('ami-071ed35cc0c7430ea', 'deep_learning_gpu_2_mine'),
 ('ami-076082b5015cd07ff', 'deep gpu new'),
 ('ami-0ad5ae4c36e1e80ef', 'deep gpu with open pose'),
 ('ami-0dacedf384f311eb2', 'p3_gpu')]

# let see if we can view volumes

In [10]:
volumes = client.describe_volumes()

In [11]:
volumes.keys()

dict_keys(['Volumes', 'ResponseMetadata'])

In [12]:
volumes['Volumes']

[{'Attachments': [],
  'AvailabilityZone': 'eu-west-1b',
  'CreateTime': datetime.datetime(2018, 6, 24, 8, 4, 10, 598000, tzinfo=tzutc()),
  'Encrypted': False,
  'Size': 400,
  'SnapshotId': 'snap-00d7c44b6c1f27940',
  'State': 'available',
  'VolumeId': 'vol-0cf697d8b22f79923',
  'VolumeType': 'standard'}]

In [13]:
sec = client.describe_security_groups()

In [14]:
sec.keys()

dict_keys(['SecurityGroups', 'ResponseMetadata'])

In [15]:
sec['SecurityGroups'][0]

{'Description': 'launch-wizard-5 created 2018-06-01T23:26:09.911+01:00',
 'GroupId': 'sg-0123fe0138e057052',
 'GroupName': 'deep-learning',
 'IpPermissions': [{'FromPort': 8888,
   'IpProtocol': 'tcp',
   'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
   'Ipv6Ranges': [],
   'PrefixListIds': [],
   'ToPort': 8888,
   'UserIdGroupPairs': []},
  {'FromPort': 22,
   'IpProtocol': 'tcp',
   'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
   'Ipv6Ranges': [],
   'PrefixListIds': [],
   'ToPort': 22,
   'UserIdGroupPairs': []}],
 'IpPermissionsEgress': [{'IpProtocol': '-1',
   'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
   'Ipv6Ranges': [],
   'PrefixListIds': [],
   'UserIdGroupPairs': []}],
 'OwnerId': '272317939526',
 'VpcId': 'vpc-cc3740aa'}

# Launch an instance

In [16]:
import random
str(random.randint(0,1000000))

'244192'

In [17]:
def launch_spot_instance(limit_price, ami, key_name, instance_type, security_group_id, availability_zone):
    try:
        response = client.request_spot_instances(
            DryRun=False,
            SpotPrice=limit_price,
            ClientToken=str(random.randint(0,1000000)),
            InstanceCount=1,
            Type='one-time',
            LaunchSpecification={
                'ImageId': ami,
                'KeyName': key_name,
                'InstanceType': instance_type,
                'Placement': {
                    'AvailabilityZone': availability_zone,
                },
                'EbsOptimized': False,
                'Monitoring': {
                    'Enabled': False
                },
                'SecurityGroupIds': [
                    security_group_id ,
                ]
            }
        )
        return response
    except Exception as e:
        raise e

In [18]:
response = launch_spot_instance('1.0', ami='ami-0ad5ae4c36e1e80ef', key_name='crispy2', security_group_id='sg-0123fe0138e057052', \
                    instance_type='p2.xlarge', availability_zone='eu-west-1b')

In [19]:
response

{'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Tue, 10 Jul 2018 06:50:07 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': '3c8b9db4-cdb5-40e3-b0c6-9b24866a4fdf',
  'RetryAttempts': 0},
 'SpotInstanceRequests': [{'CreateTime': datetime.datetime(2018, 7, 10, 6, 50, 7, tzinfo=tzutc()),
   'InstanceInterruptionBehavior': 'terminate',
   'LaunchSpecification': {'EbsOptimized': False,
    'ImageId': 'ami-0ad5ae4c36e1e80ef',
    'InstanceType': 'p2.xlarge',
    'KeyName': 'crispy2',
    'Monitoring': {'Enabled': False},
    'Placement': {'AvailabilityZone': 'eu-west-1b'},
    'SecurityGroups': [{'GroupId': 'sg-0123fe0138e057052',
      'GroupName': 'deep-learning'}],
    'SubnetId': 'subnet-65996f3f'},
   'ProductDescription': 'Linux/UNIX',
   'SpotInstanceRequestId': 'sir-cf2g6frm',
   'SpotPrice': '1.000000',
   'State': 'open',
   'Status': {'Code': 'pending

In [79]:
client.describe_spot_instance_requests(SpotInstanceRequestIds=['sir-rb1g49im'])

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '1916',
   'content-type': 'text/xml;charset=UTF-8',
   'date': 'Mon, 09 Jul 2018 09:57:06 GMT',
   'server': 'AmazonEC2'},
  'HTTPStatusCode': 200,
  'RequestId': '3d0acd6f-8e9d-4929-a52b-e165e5f0ca21',
  'RetryAttempts': 0},
 'SpotInstanceRequests': [{'CreateTime': datetime.datetime(2018, 7, 9, 9, 56, 47, tzinfo=tzutc()),
   'InstanceId': 'i-0718e02cbe9d4fa83',
   'InstanceInterruptionBehavior': 'terminate',
   'LaunchSpecification': {'EbsOptimized': False,
    'ImageId': 'ami-051fd589759f16b6c',
    'InstanceType': 'm5d.large',
    'KeyName': 'crispy2',
    'Monitoring': {'Enabled': False},
    'Placement': {'AvailabilityZone': 'eu-west-1b'},
    'SecurityGroups': [{'GroupId': 'sg-0123fe0138e057052',
      'GroupName': 'deep-learning'}],
    'SubnetId': 'subnet-65996f3f'},
   'LaunchedAvailabilityZone': 'eu-west-1b',
   'ProductDescription': 'Linux/UNIX',
   'SpotInstanceRequestId': 'sir-rb1g49im',
   'SpotPrice': '0.100000',
 

# lets see if we can mount a volume

In [80]:
response = client.attach_volume(Device='/dev/xvdh', InstanceId='i-0718e02cbe9d4fa83', VolumeId='vol-0cf697d8b22f79923')

In [81]:
response

{'AttachTime': datetime.datetime(2018, 7, 9, 9, 57, 29, 423000, tzinfo=tzutc()),
 'Device': '/dev/xvdh',
 'InstanceId': 'i-0718e02cbe9d4fa83',
 'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Mon, 09 Jul 2018 09:57:28 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': 'b8a8b2ca-757a-4ce8-917c-ca55ac32e062',
  'RetryAttempts': 0},
 'State': 'attaching',
 'VolumeId': 'vol-0cf697d8b22f79923'}

In [82]:
client.describe_volumes(VolumeIds=['vol-0cf697d8b22f79923'])

{'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Mon, 09 Jul 2018 09:57:33 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': '6912e8f0-76cb-41f0-9363-7f0ff8b43efb',
  'RetryAttempts': 0},
 'Volumes': [{'Attachments': [{'AttachTime': datetime.datetime(2018, 7, 9, 9, 57, 29, tzinfo=tzutc()),
     'DeleteOnTermination': False,
     'Device': '/dev/xvdh',
     'InstanceId': 'i-0718e02cbe9d4fa83',
     'State': 'attached',
     'VolumeId': 'vol-0cf697d8b22f79923'}],
   'AvailabilityZone': 'eu-west-1b',
   'CreateTime': datetime.datetime(2018, 6, 24, 8, 4, 10, 598000, tzinfo=tzutc()),
   'Encrypted': False,
   'Size': 400,
   'SnapshotId': 'snap-00d7c44b6c1f27940',
   'State': 'in-use',
   'VolumeId': 'vol-0cf697d8b22f79923',
   'VolumeType': 'standard'}]}

# now lets start up jupyter notebook in a desired location

In [83]:
ssm_client = boto3.client('ssm')

In [145]:
resp = ssm_client.send_command(
    DocumentName="AWS-RunShellScript", # One of AWS' preconfigured documents
    Parameters={'commands': ['cd /data/su_buildup', 'sudo -H -u ubuntu jupyter notebook &']},
    InstanceIds=['i-0718e02cbe9d4fa83']
)

In [146]:
resp['Command']['CommandId']

'3a253ee4-5611-4c92-9e28-c971c98bda16'

In [167]:
cmi = ssm_client.get_command_invocation(CommandId=resp['Command']['CommandId'], InstanceId='i-0718e02cbe9d4fa83')

In [168]:
cmi

{'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '',
  'CloudWatchOutputEnabled': False},
 'CommandId': '3a253ee4-5611-4c92-9e28-c971c98bda16',
 'Comment': '',
 'DocumentName': 'AWS-RunShellScript',
 'DocumentVersion': '',
 'ExecutionEndDateTime': '',
 'InstanceId': 'i-0718e02cbe9d4fa83',
 'PluginName': 'aws:runShellScript',
 'ResponseCode': -1,
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '471',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Mon, 09 Jul 2018 10:55:14 GMT',
   'x-amzn-requestid': '542cc7ba-ce5b-4620-b480-ef94b416dcda'},
  'HTTPStatusCode': 200,
  'RequestId': '542cc7ba-ce5b-4620-b480-ef94b416dcda',
  'RetryAttempts': 0},
 'StandardErrorContent': '',
 'StandardErrorUrl': '',
 'StandardOutputContent': '',
 'StandardOutputUrl': '',
 'Status': 'InProgress',
 'StatusDetails': 'InProgress'}